In [1]:
"""
    Get cleaned data
"""

import numpy as np

dataset = np.load('./datasets/EEGMap/EEGMapTimeseries_10F_3S.npy')
labels = np.load('./datasets/EEGMap/EEGMapTimeseries_10F_3S_Labels.npy')
print(dataset.shape)
print(labels.shape)

dataset = dataset.reshape(dataset.shape[0], dataset.shape[1], -1)
print(dataset.shape)

(576, 10, 24, 24, 3)
(576,)
(576, 10, 1728)


In [2]:
"""
    Split dataset
"""

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
shuf_dataset, shuf_labels = shuffle(dataset, labels)
X_train, X_test, y_train, y_test = train_test_split(shuf_dataset, shuf_labels, test_size=0.3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)
print("Train set size: {}, Val set size: {}, Test set size: {}".format(len(X_train), len(X_val), len(X_test)))
print(y_train.shape)
print(X_train.shape)

Train set size: 403, Val set size: 87, Test set size: 86
(403,)
(403, 10, 1728)


In [3]:
"""
    Model and train
"""

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

model_path = './models/'
json_name = 'RNN_EEGMapTimeseries_3S.json'
weight_name = 'RNN_EEGMapTimeseries_3S_tmp.h5'

batch_size = 16
epochs = 128
num_classes = 2

print("Constructing MLP Model: ...", end='')
model = Sequential()
model.add(LSTM(32, return_sequences=False, input_shape=(X_train.shape[1:])))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
print(" Complete.")

print("Compiling MLP Model: ...", end='')
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
print(" Complete.")

earlyStop = EarlyStopping(monitor='val_loss', patience=32, mode='min')
mcp_save = ModelCheckpoint(filepath=model_path+weight_name, save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')

print("Fitting MLP Model: ...")
results = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[earlyStop, mcp_save, reduce_lr_loss], validation_data=(X_val, y_val), shuffle=True)
print("Complete.")

Using TensorFlow backend.


Constructing MLP Model: ... Complete.
Compiling MLP Model: ... Complete.
Fitting MLP Model: ...
Train on 403 samples, validate on 87 samples
Epoch 1/128
403/403 [==============================] - 2s 5ms/step - loss: 0.6994 - acc: 0.6005 - val_loss: 0.6320 - val_acc: 0.7356
Epoch 2/128
403/403 [==============================] - 1s 1ms/step - loss: 0.6868 - acc: 0.6179 - val_loss: 0.6072 - val_acc: 0.6552
Epoch 3/128
403/403 [==============================] - 1s 1ms/step - loss: 0.6696 - acc: 0.6278 - val_loss: 0.6712 - val_acc: 0.6897
Epoch 4/128
403/403 [==============================] - 1s 1ms/step - loss: 0.6616 - acc: 0.6228 - val_loss: 0.6323 - val_acc: 0.6552
Epoch 5/128
403/403 [==============================] - 1s 1ms/step - loss: 0.6378 - acc: 0.6501 - val_loss: 0.6283 - val_acc: 0.6552
Epoch 6/128
403/403 [==============================] - 1s 1ms/step - loss: 0.6275 - acc: 0.6551 - val_loss: 0.5843 - val_acc: 0.7011
Epoch 7/128
403/403 [==============================] - 1s 1ms

In [4]:
"""
    Save the model
"""

model_path = './models/'
json_name = 'RNN_EEGMapTimeseries_3S.json'
weight_name = 'RNN_EEGMapTimeseries_3S.h5'

model_json = model.to_json()
with open(model_path+json_name, 'w') as json_file:
    json_file.write(model_json)
#model.save_weights(model_path+weight_name)

In [5]:
"""
    Output & plot the results
    Aspect Ratio 2 to 1
"""

import matplotlib.pyplot as plt

image_path = './graphs/'
image_name = 'RNN_EEGMapTimeseries3S_2to1.png'

fig = plt.figure(figsize=(10,5))
plt.plot(results.history['loss'], label='train loss')
plt.plot(results.history['val_loss'], label='val loss')
plt.plot(results.history['acc'], label='train acc')
plt.plot(results.history['val_acc'], label='val acc')
#plt.ylim((0.05,1.05))
plt.legend(loc='lower left')
plt.title('AF7, AF8, TP9, TP10 EEG Map Timeseries 3S [1-50 Hz] RNN')
plt.xlabel('Epoch(s)')
plt.ylabel('Performance')
#plt.show()
plt.savefig(image_path+image_name, bbox_inches='tight')
test_scores = model.evaluate(X_test, y_test)
print("\nFinal test %s: %.2f%%" % (model.metrics_names[1], test_scores[1]*100))

86/86 [==============================] - 0s 589us/step

Final test acc: 88.37%


In [ ]:
"""
    Output & plot the results
    Aspect Ratio 2 to 1
"""

import matplotlib.pyplot as plt

image_path = './graphs/'
image_name = 'RNN_EEGMapTimeseries3S_2to1.png'

fig = plt.figure(figsize=(16,9))
plt.plot(results.history['loss'], label='train loss')
plt.plot(results.history['val_loss'], label='val loss')
plt.plot(results.history['acc'], label='train acc')
plt.plot(results.history['val_acc'], label='val acc')
plt.legend(loc='lower left')
plt.title('AF7, AF8, TP9, TP10 EEG Map Timeseries 3S [1-50 Hz] RNN')
plt.xlabel('Epoch(s)')
plt.ylabel('Performance')
#plt.show()
plt.savefig(image_path+image_name, bbox_inches='tight')